# AMT Loader

This notebook demonstrates the AMT loader functions from `specparser.amt.loader`.

## AMT File Format

AMT files are YAML files with a specific structure:
- `backtest`: Configuration values (aum, leverage)
- `amt`: Asset definitions keyed by name, each with an `Underlying` identifier
- Rule tables: Tables with `Columns` and `Rows` for classification rules

In [ ]:
# Setup: Imports and helper function
import tempfile
import os
import pandas as pd
from specparser.amt import (
    load_amt,
    clear_cache,
    get_value,
    get_aum,
    get_leverage,
    get_asset,
    find_assets,
    assets,
    cached_assets,
    _iter_assets,
    asset_class,
    get_table,
    _compile_rules,
    _match_rules,
    asset_table,
    asset_group,
    table_to_rows,
)

def show(table):
    """Display a table as a pandas DataFrame for pretty rendering."""
    t = table_to_rows(table) if table.get("orientation") == "column" else table
    return pd.DataFrame(t["rows"], columns=t["columns"])

In [ ]:
# Create a temporary AMT file for all examples
test_amt_content = """
backtest:
  aum: 1000000.0
  leverage: 2.5

amt:
  Apple:
    Underlying: "AAPL US Equity"
    Class: "Equity"
    WeightCap: 0.10
    Vol:
      Source: "BBG"
    Hedge:
      Source: "BBG"
    Valuation:
      Model: "BS"

  Google:
    Underlying: "GOOGL US Equity"
    Class: "Equity"
    WeightCap: 0.08
    Vol:
      Source: "BBG"
    Hedge:
      Source: "BBG"
    Valuation:
      Model: "BS"

  CrudeOil:
    Underlying: "CL1 Comdty"
    Class: "Commodity"
    WeightCap: 0.05
    Vol:
      Source: "Internal"
    Hedge:
      Source: "BBG"
    Valuation:
      Model: "Bachelier"

  Gold:
    Underlying: "GC1 Comdty"
    Class: "Commodity"
    WeightCap: 0.0
    Vol:
      Source: "BBG"
    Hedge:
      Source: "BBG"

  TenYear:
    Underlying: "TY1 Comdty"
    Class: "Rate"
    WeightCap: 0.15
    Vol:
      Source: "Internal"
    Hedge:
      Source: "Internal"
    Valuation:
      Model: "Bachelier"

group_table:
  Columns: [field, rgx, value]
  Rows:
    - [Class, "^Equity$", "equities"]
    - [Class, "^Commodity$", "commodities"]
    - [Class, "^Rate$", "rates"]
    - [Class, ".*", "other"]

subgroup_table:
  Columns: [field, rgx, value]
  Rows:
    - [Underlying, ".*Equity$", "stocks"]
    - [Underlying, ".*Comdty$", "futures"]
    - [Underlying, ".*", ""]

liquidity_table:
  Columns: [field, rgx, value]
  Rows:
    - [Class, "^Equity$", "high"]
    - [Class, "^Commodity$", "medium"]
    - [Class, ".*", "low"]

limit_overrides:
  Columns: [field, rgx, value]
  Rows:
    - [Underlying, "^CL1", "0.03"]
    - [Underlying, ".*", ""]
"""

# Write to temp file
temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.yml', delete=False)
temp_file.write(test_amt_content)
temp_file.close()
AMT_PATH = temp_file.name

print(f"Created test AMT file: {AMT_PATH}")
print(f"\nTest data includes:")
print(f"  - 5 assets (4 live, 1 not live)")
print(f"  - 3 asset classes: Equity, Commodity, Rate")
print(f"  - 4 rule tables for grouping")

---
## 1. File Loading & Caching

### `load_amt(path)`

Load an AMT YAML file and return the full parsed data. Results are cached.

In [ ]:
# Load the AMT file
data = load_amt(AMT_PATH)

print(f"Top-level keys: {list(data.keys())}")
print(f"Number of assets: {len(data['amt'])}")
print(f"Asset names: {list(data['amt'].keys())}")

### `clear_cache()`

Clear the AMT file cache. Useful when the file has been modified.

In [ ]:
# Clear and reload to demonstrate caching
clear_cache()
data = load_amt(AMT_PATH)  # This reloads from disk
print("Cache cleared and file reloaded")

---
## 2. Value Access

### `get_value(path, key_path, default)`

Get a value from an AMT file by its dot-separated key path.

In [ ]:
# Get nested values using dot notation
aum = get_value(AMT_PATH, "backtest.aum")
leverage = get_value(AMT_PATH, "backtest.leverage")

print(f"backtest.aum = {aum}")
print(f"backtest.leverage = {leverage}")

In [ ]:
# Default value for missing keys
missing = get_value(AMT_PATH, "nonexistent.key", default="NOT FOUND")
print(f"Missing key with default: {missing}")

### `get_aum(path)` and `get_leverage(path)`

Convenience functions for common backtest values.

In [ ]:
print(f"AUM: {get_aum(AMT_PATH):,.0f}")
print(f"Leverage: {get_leverage(AMT_PATH)}")

---
## 3. Asset Queries

### `get_asset(path, underlying)`

Get asset data by its Underlying value.

In [ ]:
# Get a specific asset
apple = get_asset(AMT_PATH, "AAPL US Equity")

print("Apple asset data:")
for key, value in apple.items():
    print(f"  {key}: {value}")

In [ ]:
# Asset not found returns None
missing = get_asset(AMT_PATH, "NONEXISTENT")
print(f"Missing asset: {missing}")

### `find_assets(path, pattern, live_only)`

Find all Underlying values matching a regex pattern.

In [ ]:
# Find all equity assets
equities = find_assets(AMT_PATH, "Equity$")
show(equities)

In [ ]:
# Find commodity assets that are live (WeightCap > 0)
live_commodities = find_assets(AMT_PATH, "Comdty$", live_only=True)
print("Live commodities (WeightCap > 0):")
show(live_commodities)

In [ ]:
# No matches returns empty table
no_match = find_assets(AMT_PATH, "^NOMATCH")
print(f"No matches: {len(no_match['rows'])} rows")
show(no_match)

### `assets(path, live_only, pattern)`

Get assets with their Underlying values.

In [ ]:
# All assets
all_assets = assets(AMT_PATH)
print(f"All assets ({len(all_assets['rows'])}):")
show(all_assets)

In [ ]:
# Live assets only
live_assets = assets(AMT_PATH, live_only=True)
print(f"Live assets ({len(live_assets['rows'])}):")
show(live_assets)

### `cached_assets(path)`

List all asset Underlying values from the cache.

In [ ]:
# Get assets from cache (faster, no regex matching)
cached = cached_assets(AMT_PATH)
show(cached)

### `_iter_assets(path, live_only, pattern)`

Iterator over assets. Yields `(asset_data, underlying)` tuples.

In [ ]:
# Iterate over live assets
print("Live assets with their classes:")
for asset_data, underlying in _iter_assets(AMT_PATH, live_only=True):
    print(f"  {underlying}: {asset_data.get('Class')}")

---
## 4. Asset Classification

### `asset_class(path, live_only, pattern)`

Get assets with their class, vol source, hedge source, and valuation model.

In [ ]:
# All assets with classification info
classification = asset_class(AMT_PATH)
show(classification)

In [ ]:
# Live assets only
live_classification = asset_class(AMT_PATH, live_only=True)
show(live_classification)

---
## 5. Embedded Tables

### `get_table(path, key_path)`

Get an embedded table from an AMT file. Tables have `Columns` and `Rows` keys.

In [ ]:
# Get the group_table
group_rules = get_table(AMT_PATH, "group_table")
print(f"Columns: {group_rules['columns']}")
show(group_rules)

In [ ]:
# Error case: table not found
try:
    get_table(AMT_PATH, "nonexistent_table")
except ValueError as e:
    print(f"Error: {e}")

---
## 6. Rule Matching

### `_compile_rules(table)`

Compile rules from a table with columns `[field, rgx, value]`.

In [ ]:
# Compile the group rules
rules = _compile_rules(group_rules)

print("Compiled rules (field, pattern, value):")
for field, pattern, value in rules:
    print(f"  {field}: {pattern.pattern!r} -> {value!r}")

### `_match_rules(rules, field_values, default)`

Find the first matching rule and return its value.

In [ ]:
# Match against compiled rules
result1 = _match_rules(rules, {"Class": "Equity", "Underlying": "AAPL US Equity"})
result2 = _match_rules(rules, {"Class": "Commodity", "Underlying": "CL1 Comdty"})
result3 = _match_rules(rules, {"Class": "Unknown", "Underlying": "XXX"})

print(f"Equity -> {result1}")
print(f"Commodity -> {result2}")
print(f"Unknown -> {result3} (caught by .* fallback)")

### `asset_table(path, table_name, default, live_only, pattern)`

Evaluate a classification rule table against assets.

In [ ]:
# Evaluate group_table against live assets
groups = asset_table(AMT_PATH, "group_table", live_only=True)
show(groups)

In [ ]:
# Evaluate limit_overrides (only CL1 has an override)
limits = asset_table(AMT_PATH, "limit_overrides", default="", live_only=True)
show(limits)

---
## 7. Asset Grouping

### `asset_group(path, live_only, pattern)`

Get assets with group, subgroup, liquidity, and limit override columns.

This combines multiple rule tables into one result.

In [ ]:
# Full asset grouping for live assets
full_grouping = asset_group(AMT_PATH, live_only=True)
show(full_grouping)

In [ ]:
# Filter to specific pattern
commodity_grouping = asset_group(AMT_PATH, live_only=True, pattern="Comdty$")
show(commodity_grouping)

---
## Cleanup

In [ ]:
# Remove temporary file and clear cache
os.unlink(AMT_PATH)
clear_cache()
print("Cleanup complete: temp file removed, cache cleared")

---
## Summary

| Function | Description |
|----------|-------------|
| `load_amt` | Load and cache AMT YAML file |
| `clear_cache` | Clear the file cache |
| `get_value` | Get value by dot-separated key path |
| `get_aum` | Get AUM from backtest section |
| `get_leverage` | Get leverage from backtest section |
| `get_asset` | Get single asset by Underlying |
| `find_assets` | Find assets by regex pattern |
| `assets` | List all assets |
| `cached_assets` | List assets from cache |
| `_iter_assets` | Iterator over assets |
| `asset_class` | Assets with class/source info |
| `get_table` | Get embedded table by key path |
| `_compile_rules` | Compile regex rules from table |
| `_match_rules` | Match field values against rules |
| `asset_table` | Evaluate rule table against assets |
| `asset_group` | Assets with full grouping info |